In [1]:
import findspark
findspark.init('/home/aditya/spark-3.1.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Lreg").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [6]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
# df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [8]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
### setting data for machine learning in pyspark

### here we first take all the numerical values to create data for building model
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.feature import StringIndexer

In [10]:
# indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_lineIndex")
# indexed = indexer.fit(df).transform(df)
# indexed.head(1)

In [11]:
# assempler = VectorAssembler(inputCols=[ 'Age',
#  'Tonnage',
#  'passengers',
#  'length',
#  'cabins',
#  'passenger_density','Cruise_lineIndex'], outputCol='features')

In [12]:
# finaldata = assempler.transform(indexed)

In [13]:
# finalreldata = finaldata.select(['features','crew'])
# finalreldata.show()

In [14]:
##split data into train and test

train_data, test_data = df.randomSplit([0.7,0.3])

In [15]:
train_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[95,96,97,12...|
|  0.0|(692,[98,99,100,1...|
|  0.0|(692,[100,101,102...|
|  0.0|(692,[122,123,148...|
|  0.0|(692,[123,124,125...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[125,126,127...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[126,127,128...|
|  0.0|(692,[127,128,129...|
|  0.0|(692,[127,128,129...|
+-----+--------------------+
only showing top 20 rows



In [16]:
lr = LogisticRegression(featuresCol='features',labelCol='label',predictionCol='predicted')

In [17]:
model = lr.fit(train_data)

In [18]:
model.coefficients

SparseVector(692, {95: 0.0057, 96: 0.0013, 97: 0.0048, 98: 0.0048, 99: 0.0032, 100: -0.0077, 101: -0.0107, 102: -0.0115, 119: 0.0216, 120: 0.0073, 121: 0.0139, 122: -0.0075, 123: 0.0, 124: 0.0022, 125: 0.0018, 126: 0.0011, 127: 0.0014, 128: 0.0013, 129: -0.0012, 130: 0.0005, 131: 0.0023, 132: 0.0032, 133: 0.0048, 146: 0.0224, 147: 0.0076, 148: 0.0061, 149: 0.0033, 150: 0.0009, 151: -0.0005, 152: 0.0004, 153: -0.0004, 154: -0.001, 155: -0.0011, 156: -0.0012, 157: -0.0008, 158: 0.0002, 159: 0.0015, 160: 0.0016, 161: 0.0, 162: 0.0017, 163: 0.0035, 164: 0.0214, 174: 0.0105, 175: 0.0047, 176: 0.0021, 177: 0.0007, 178: 0.0006, 179: -0.0002, 180: -0.0012, 181: -0.0, 182: 0.0008, 183: 0.0018, 184: -0.0002, 185: -0.0008, 186: 0.0002, 187: 0.0006, 188: -0.0001, 189: -0.0004, 190: 0.0025, 191: 0.0012, 192: 0.0021, 202: -0.0051, 203: 0.0009, 204: 0.0001, 205: 0.0007, 206: 0.0012, 207: -0.001, 208: -0.0011, 209: -0.0002, 210: 0.002, 211: 0.0027, 212: 0.0005, 213: -0.0012, 214: -0.0006, 215: 0.0001,

In [19]:
model.intercept

0.3325580865398148

In [20]:
model_summary = model.summary

In [22]:
model_summary.accuracy

1.0

In [23]:
model_summary.areaUnderROC

1.0

In [24]:
test_results = model.evaluate(test_data)

In [36]:
test_results.predictions

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, predicted: double]

In [25]:
test_results.areaUnderROC

1.0

In [26]:
##deployment / inferencing - nolables

unlabeldata = test_data.select('features')
unlabeldata.show()

+--------------------+
|            features|
+--------------------+
|(692,[121,122,123...|
|(692,[122,123,124...|
|(692,[123,124,125...|
|(692,[123,124,125...|
|(692,[124,125,126...|
|(692,[126,127,128...|
|(692,[151,152,153...|
|(692,[153,154,155...|
|(692,[99,100,101,...|
|(692,[123,124,125...|
|(692,[123,124,125...|
|(692,[124,125,126...|
|(692,[124,125,126...|
|(692,[127,128,129...|
|(692,[127,128,129...|
|(692,[128,129,130...|
|(692,[128,129,130...|
|(692,[128,129,130...|
|(692,[129,130,131...|
|(692,[156,157,158...|
+--------------------+
only showing top 20 rows



In [27]:
unlabeldata_predicted = model.transform(unlabeldata)
unlabeldata_predicted.show()

+--------------------+--------------------+--------------------+---------+
|            features|       rawPrediction|         probability|predicted|
+--------------------+--------------------+--------------------+---------+
|(692,[121,122,123...|[20.8019710258389...|[0.99999999907568...|      0.0|
|(692,[122,123,124...|[16.2625004637160...|[0.99999991344627...|      0.0|
|(692,[123,124,125...|[28.2831558986001...|[0.99999999999947...|      0.0|
|(692,[123,124,125...|[29.6653606308125...|[0.99999999999986...|      0.0|
|(692,[124,125,126...|[24.2992929477983...|[0.99999999997201...|      0.0|
|(692,[126,127,128...|[16.1633594614495...|[0.99999990442547...|      0.0|
|(692,[151,152,153...|[18.6507967963425...|[0.99999999205558...|      0.0|
|(692,[153,154,155...|[9.69529197500039...|[0.99993843109142...|      0.0|
|(692,[99,100,101,...|[-8.6297662847624...|[1.78674476421406...|      1.0|
|(692,[123,124,125...|[-24.351960477264...|[2.65508201673799...|      1.0|
|(692,[123,124,125...|[-1

In [28]:
## evaluationg the model

In [38]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [42]:
my_evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label')

In [44]:
my_final_roc = my_evaluator.evaluate(test_results.predictions)

In [45]:
my_final_roc

1.0